# 1.1 Необходимо с помощью SQL запроса вывести общий календарный Retention первого дня

Retention rate - коэф удержания (пользователи в конце периода / пользователи в начале периода) #пользователи одни и те же <br/>
При расчете календарного retention, например, 1-го дня, учитывается вход в интервале от 0 до 48 часов с момента установки

In [ ]:
# 90 минут найти способ как перенести таблицы в psql (до этого делал в sqlite3)

# 20 минут читать и вникать в retantion 1 дня

# 62 минуты на первое sql задание

"""

SELECT 
	
	CONCAT(ROUND((count(day_one_users)/count(day_zero_users)::float*100)::numeric, 3), '%') as day_one_retention
	
FROM
	(SELECT 
	DISTINCT id,
	CASE WHEN diff < time_before_next_day THEN 'zero' END AS day_zero_users,
	CASE WHEN (diff > time_before_next_day AND diff < time_before_next_day+24) THEN 'one' END AS day_one_users

	FROM(
		SELECT StartGame.DeviceId as id,
		EXTRACT(EPOCH FROM StartGame.event_datetime - FirstOpen.event_datetime)/3600 as diff,
		EXTRACT(EPOCH FROM (date (FirstOpen.event_datetime) +1 - FirstOpen.event_datetime))/3600 as time_before_next_day

		FROM FirstOpen
		JOIN StartGame on FirstOpen.DeviceId = StartGame.deviceid
		) tmp )tmp
		
WHERE (day_zero_users <> 'NULL' OR day_one_users <> 'NULL')

"""

-> 33.385%

# 1.2 Необходимо с помощью SQL вместе с Retention первого дня вывести среднее количество сессий, наигранных игроками, попавших в Retention первого дня, за два дня.


In [ ]:
# 35 минут на второе задание

"""

SELECT 
	CONCAT(max(day_one_retention), '%') as retention_первого_дня, 
	round(max(avg), 3) as среднее_количество_сессий

FROM 
	(SELECT 
			avg(sessions_count)
		FROM
			(SELECT 
			 count(*) FILTER (WHERE diff < time_before_next_day+24) OVER(PARTITION BY id) as sessions_count,
			CASE WHEN diff < time_before_next_day THEN 'zero' END AS day_zero_users,
			CASE WHEN (diff > time_before_next_day AND diff < time_before_next_day+24) THEN 'one' END AS day_one_users

			FROM(
				SELECT StartGame.DeviceId as id, 
				StartGame.event_datetime as event_time,

				EXTRACT(EPOCH FROM StartGame.event_datetime - FirstOpen.event_datetime)/3600 as diff,
				EXTRACT(EPOCH FROM (date (FirstOpen.event_datetime) +1 - FirstOpen.event_datetime))/3600 as time_before_next_day

				FROM FirstOpen
				JOIN StartGame on FirstOpen.DeviceId = StartGame.deviceid
				) tmp 
			)tmp
		WHERE (day_zero_users <> 'NULL' OR day_one_users <> 'NULL') )tmp
		
FULL JOIN 
	(SELECT 
	
	ROUND((count(day_one_users)/count(day_zero_users)::float*100)::numeric, 3) as day_one_retention
	
	FROM
		(SELECT 
		DISTINCT id,
		CASE WHEN diff < time_before_next_day THEN 'zero' END AS day_zero_users,
		CASE WHEN (diff > time_before_next_day AND diff < time_before_next_day+24) THEN 'one' END AS day_one_users

		FROM(
			SELECT StartGame.DeviceId as id,

			EXTRACT(EPOCH FROM StartGame.event_datetime - FirstOpen.event_datetime)/3600 as diff,
			EXTRACT(EPOCH FROM (date (FirstOpen.event_datetime) +1 - FirstOpen.event_datetime))/3600 as time_before_next_day

			FROM FirstOpen
			JOIN StartGame on FirstOpen.DeviceId = StartGame.deviceid
			) tmp )tmp

		WHERE (day_zero_users <> 'NULL' OR day_one_users <> 'NULL')
	) as ret_table on ret_table.day_one_retention = tmp.avg
    
"""

-> 33.385% | 3.614

# 2. 
Представим ситуацию, что был запущен A/B тест, в котором первая A/B группа играет на основной версии игры, вторая A/B группа играет на версии с измененным туториалом. <br/>
 Группы катились три дня, к Вам подходит продюсер проекта и говорит, что он заметил сильную разницу в общем доходе между группами и беспокоится, <br/>
 что тестовая группа создает просадку по доходу, поэтому думает, что тест надо заканчивать. <br/>
 Ваша задача : при помощи данных в таблице дать аргументированное решение по данному A/B тесту.


In [ ]:
# 28 минут

"""

Прерывать A/B тест не дождавшись окончания в принципе не лучшая идея
Такая штука называется проблемой подглядывания и кратно увеличивает вероятность совершения ошибки

Есть два основных способа досрочного завершения, но они оба подразумевают, что заранее оговорено количество промежуточных проверок

Граница Покока : выбрать количество "подглядываний", поделить принятый нами порог p-value на это количество подглядываний,
 и останавливаться только в том случае, если мы упали за получившийся порог.

O'Brien-Fleming - имперически найденная формула, где p-value тем меньше, чем дальше мы от окончания теста находимся. 
Проще нагуглить или формулу, или python код

Так или иначе, ориентируясь на paying share, калькуляторы говорят, что значимых различий между группами нет,
 и потому как проверки не обозначены, мы не должны останавливать проведение теста


"""

# 3. 
В Ваши задачи, как аналитика, входит работа по двум играм: <br/>
Первый проект имеет метрики Paying share 2% и ARPPU 30$ <br/>
Второй  проект имеет метрики Paying share 4% и ARPPU 13$ <br/>

Такие метрики достигаются на 10000 пользователей на каждой игре за одни и те же периоды.

Какая игра, по Вашему мнению, приносит больше прибыли при равных условиях и почему? Чем, с точки зрения геймдизайна, экономики, баланса и т.д., могут отличаться такие игры между собой? Какие еще различия в метриках могут быть у таких игр?



In [ ]:
# 24 минуты

""" 

Конкретно в данном примере чтобы узнать сколько выручки принесла игра, нужно пойти в обратную сторону от arppu (выручка/платящие пользователи)
 и умножить платящих пользователей на среднюю выручку
Таким образом, с платежей пользователей, первая игра принесла 6000, вторая 5200

К тому же возможно первая игра имеет больше возможности получить что-то за просмотры рекламы и таким образом может 
приносить больший доход еще и за этот счет, что к тому же объяснит больший arppu и меньший paying share 


Чем, с точки зрения геймдизайна, экономики, баланса и т.д., могут отличаться такие игры между собой?

Ссылаясь на предыдущее предположение, предположу, что есть какой-то порог по сумме, 
который уже нельзя перешагнуть просмотрами рекламы, и приходится платить

Учитывая, что средний платеж в первой игре выше, а платящих пользователей меньше, скорее всего предметы покупки там дают 
большее преимущество над бесплатными или за просмотр рекламы, а так же имеют более высокие цены



Какие еще различия в метриках могут быть у таких игр?

Ссылаясь на возможные просмотры рекламы или другие способы проще получить какие-то платные предметы, скорее всего в первой выше retention, 
выше количество пользователей онлайн, больше времени в среднем проводит пользователь в игре, больше сессий в среднем, 
больше просмотров рекламы, выше средний чек

У второй игры выше конверсия в первый платеж

"""